In [ ]:
import os
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import scipy
import matplotlib as mpl

from type3detect import detectRadioburst as drb

%load_ext autoreload
%autoreload 2

In [ ]:
fname  = './LOFAR_20220701_070000_LBA_OUTER_S0.fits'

(dyspec,t_fits,f_fits,hdu)  = drb.read_fits(fname)
(dyspec,f_fits) =  drb.cut_low(dyspec,f_fits,f_low_cut_val=30)

In [ ]:
(data_fits_new,data_fits_new_smooth) = drb.preproc(dyspec,gauss_sigma=1.5)

fig = plt.figure(figsize=(6, 4), dpi=120)
ax = plt.gca()

data_safe_arr = data_fits_new_smooth.ravel()
data_safe = np.sort(data_safe_arr)[int(data_safe_arr.shape[0] * 0.02):int(data_safe_arr.shape[0] * 0.98)]
vmin,vmax = [(np.nanmean(data_safe) - 2 * np.nanstd(data_safe)),
                  (np.nanmean(data_safe) + 4 * np.nanstd(data_safe)+0.5*np.nanmax(data_safe))]
        
ax.imshow(data_fits_new_smooth.T,aspect='auto',  origin='lower', vmax=vmax,vmin=vmin,
                   extent=[t_fits[0],t_fits[-1],f_fits[0],f_fits[-1]],cmap='inferno')

ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.set_title(hdu[0].header['CONTENT'])

## Binarization Local max 

Threshold: $$I> I_{thresh}$$

Local-max: $$ I_{i-1}< I_{i}> I_{i+1} $$

Second order Local-max: $$ I_{i-2}< I_{i-1}< I_{i}> I_{i+1}> I_{i+2} $$

Second order Local-max with ratio R=1.2: $$ R^2 \times I_{i-2}< R \times I_{i-1}< I_{i}> R\times I_{i+1}> R^2 \times I_{i+2} $$


In [ ]:
# binarization
bmap = drb.binarization(data_fits_new_smooth,N_order=6,peak_r=1.002)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[6,3],dpi=200)
ax.imshow(1-bmap.T,aspect='auto', origin='lower',cmap='gray')

In [ ]:
fig = plt.figure(figsize=[6,3],dpi=200)
plt.imshow(data_fits_new_smooth.T,aspect='auto',origin='lower',vmax=vmax,vmin=vmin/2-vmax/2)
plt.contour(bmap.T,[0,0.5,1],colors='k')
plt.title("overplot binary map to spectrum")

## Hough transform

Identify type-III (line-like features) in the image using the Hough transform.

In [ ]:
# detect verticle features
lines = drb.hough_detect(bmap,dyspec,threshold=40,line_gap=10,line_length=30,
            theta=np.linspace(np.pi/2-np.pi/8,np.pi/2-1/180*np.pi,300))

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[6,3],dpi=200)
lines = sorted(lines, key=lambda i: i[0][1])
ax.imshow(1-bmap.T,aspect='auto',origin='lower',cmap='gray')
for line in lines:
    p0,p1= line
    ax.plot( (p0[1], p1[1]),(p0[0], p1[0]),':')


In [ ]:
line_sets = drb.line_grouping(lines)
# group the detected lines into group in regard of events

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[6,3],dpi=200)
ax.imshow(data_fits_new_smooth.T,aspect='auto',origin='lower',vmax=vmax,vmin=vmin,cmap='gray')

for idx,lines in enumerate(line_sets):
    for line in lines:
        p0,p1= line
        ax.plot( (p0[1], p1[1]),(p0[0], p1[0]),color='C'+str(idx+1))
        ax.plot( (p0[1], p1[1]),(p0[0], p1[0]),'k+',zorder=10)
    #ax.set_xlim((500,600))
#ax.set_ylim((bmap.shape[0], 0))

## Model the electron beam

To describe the t and f points in the radio burst.

Model a electron beam going outward with a speed of v_{beam}, with a starting time $t_0$, we can have r(t).

With a density model:

$$N_e(r) = 4.8\times 10^9/r^{14} + 3\times 10^8/r^6+1.39\times 10^6/r^{2.3}+n_0 $$

And $f=8.93\times 10^3 \sqrt{N_e}$

We can have a f(t) from the model, which is the frequency drift line.

The modeling procedure is to find a combination of $[v_{beam},t_0]$ to make the output of the model f(t) fit best to the data points above

In [ ]:
from type3detect import radioTools as rt

In [ ]:
#(rt.freq_to_R(20e6)-rt.freq_to_R(80e6))/rt.c_r

In [ ]:
(v_beam, f_range_burst, t_range_burst, model_curve_set,
     t_set_arr_set,f_set_arr_set,t_model_arr,f_model_arr
    )= drb.get_info_from_linegroup(line_sets,t_fits,f_fits)

In [ ]:
plt.plot(t_set_arr_set[0],f_set_arr_set[0],'x')

In [ ]:
plt.plot(t_model_arr,f_model_arr)

In [ ]:
line_sets

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[6,3],dpi=200)
lines = sorted(lines, key=lambda i: i[0][1])
ax.imshow(data_fits_new.T,aspect='auto',origin='lower', vmax=vmax,vmin=vmin,cmap='gray',
                   extent=[t_fits[0],t_fits[-1],f_fits[0],f_fits[-1]])
for idx,model in enumerate(model_curve_set):
    if (v_beam[idx] > 0) and (v_beam[idx] < 0.9):
        plt.plot(model[0],model[1],ls='--')
        plt.plot(t_range_burst[idx],f_range_burst[idx],'k+')
    


ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.set_title(hdu[0].header['CONTENT'])


#ax.set_xlim([t_fits[200],t_fits[400]])
#plt.ylim([10,88])


In [ ]:
# dump information to Json file
import json

In [ ]:
fname_json  = fname.replace('.fits','.json')

In [ ]:
with open(fname_json, 'r') as fp:
    dict_old = json.load(fp)
fp.close()

In [ ]:
(t_range_burst[0][1]-t_range_burst[0][0])*3600*24

In [ ]:
dict_old

In [ ]:
event_detail = []
for idx,v_cur in enumerate(v_beam):
    event_detail.append({
        'v_beam':v_cur,
        'freq_range':((f_range_burst[idx])),
        'time_range':((t_range_burst[idx])),
        'str_time':mdates.num2date(t_range_burst[idx][0]).strftime("%H:%M:%S")})

In [ ]:
dict_old['event']={
    'detection': True,
    'type':'III',
    'detail': event_detail
}

In [ ]:
dict_old

In [ ]:
with open('test.json', 'w') as fp:
     json.dump(dict_old,fp)
fp.close()

In [ ]:
# write all events to a event list

import os
csv_fname = 'event.csv'
os.system('rm '+csv_fname)
id_event  = 0
with open(csv_fname,'w') as fp:
    fp.write('''ID, t, t0_num, t1_num,f_0,f_1, dfdt(MHz/s), v_b(c)
             ''')

with open(csv_fname,'a') as fp:
    
    for idx,v_cur in enumerate(v_beam):
        fp.write(str(id_event)+','+mdates.num2date(t_range_burst[idx][0]).strftime("%y-%m-%d %H:%M:%S")+','
             +str(t_range_burst[idx][0])+','+str(t_range_burst[idx][1])+','
             +str(f_range_burst[idx][0])+','+str(f_range_burst[idx][1])+','
             +str((np.max(f_set_arr_set[idx])-np.min(f_set_arr_set[idx]))/
             (np.max(t_set_arr_set[idx])-np.min(t_set_arr_set[idx])))+','
             +str(v_beam[idx])
             +'''
             ''')
        id_event+=1
fp.close()

In [ ]:

os.system('pwd')

In [ ]:
import glob

In [ ]:
glob.glob('L*')